In [40]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

In [41]:
wine_data = pd.read_csv('data/wine_data.csv')
wine_data.sample(5)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
91,1,12.00,1.51,2.42,22.0,86,1.45,1.25,0.50,1.63,3.60,1.05,2.65,450
174,2,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
148,2,13.32,3.24,2.38,21.5,92,1.93,0.76,0.45,1.25,8.42,0.55,1.62,650
10,0,14.10,2.16,2.30,18.0,105,2.95,3.32,0.22,2.38,5.75,1.25,3.17,1510
100,1,12.08,2.08,1.70,17.5,97,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710


#### Collecting Features

In [42]:
wine_features = wine_data.drop('Class', axis = 1)
wine_target = wine_data[['Class']]

In [43]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(wine_features,
                                                    wine_target,
                                                    test_size=0.4,
                                                    random_state=0)

In [44]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(x_test.values).float()

In [45]:
Ytrain_ = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest_ = torch.from_numpy(y_test.values).view(1,-1)[0]

## Creating a classifier


In [46]:
input_size = 13
output_size = 3
hidden_size = 100

In [47]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [48]:
model = Net()

In [49]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.NLLLoss()

#### Training the model

In [50]:
epochs = 1000

for epoch in range(epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    if epoch % 100 == 0:
        print ('Epoch', epoch, 'loss', loss.item())

Epoch 0 loss 1.0995593070983887
Epoch 100 loss 0.2302762269973755
Epoch 200 loss 0.0429576113820076
Epoch 300 loss 0.03152114897966385
Epoch 400 loss 0.08111901581287384
Epoch 500 loss 0.05329259857535362
Epoch 600 loss 0.050014883279800415
Epoch 700 loss 0.023205997422337532
Epoch 800 loss 0.013037456199526787
Epoch 900 loss 0.41736555099487305


## Export to ONNX
* Exporting models in PyTorch is done via tracing. This is done with the aid of the torch.onnx._export() function. This function will execute the model and record a trace of what operators are used to compute the outputs. Since _export runs the model, we need to provide an input tensor x
* Do not need to install onnx. Available as part of the pytorch package

#### Get a sample input for onnx export

In [51]:
sample = np.array(x_test)

sample_tensor = torch.from_numpy(sample).float()

In [52]:
sample_tensor

tensor([[1.3740e+01, 1.6700e+00, 2.2500e+00, 1.6400e+01, 1.1800e+02, 2.6000e+00,
         2.9000e+00, 2.1000e-01, 1.6200e+00, 5.8500e+00, 9.2000e-01, 3.2000e+00,
         1.0600e+03],
        [1.2790e+01, 2.6700e+00, 2.4800e+00, 2.2000e+01, 1.1200e+02, 1.4800e+00,
         1.3600e+00, 2.4000e-01, 1.2600e+00, 1.0800e+01, 4.8000e-01, 1.4700e+00,
         4.8000e+02],
        [1.2370e+01, 1.1300e+00, 2.1600e+00, 1.9000e+01, 8.7000e+01, 3.5000e+00,
         3.1000e+00, 1.9000e-01, 1.8700e+00, 4.4500e+00, 1.2200e+00, 2.8700e+00,
         4.2000e+02],
        [1.3560e+01, 1.7300e+00, 2.4600e+00, 2.0500e+01, 1.1600e+02, 2.9600e+00,
         2.7800e+00, 2.0000e-01, 2.4500e+00, 6.2500e+00, 9.8000e-01, 3.0300e+00,
         1.1200e+03],
        [1.3050e+01, 5.8000e+00, 2.1300e+00, 2.1500e+01, 8.6000e+01, 2.6200e+00,
         2.6500e+00, 3.0000e-01, 2.0100e+00, 2.6000e+00, 7.3000e-01, 3.1000e+00,
         3.8000e+02],
        [1.1560e+01, 2.0500e+00, 3.2300e+00, 2.8500e+01, 1.1900e+02, 3.1800e+00,

In [53]:
torch.onnx.export(model, sample_tensor, "classifier.onnx", export_params=True)

# Converting Model from PyTorch to Caffe2


## Import libraries
* onnx-caffe2 is now integrated as part of caffe2 under caffe2/python/onnx.

In [54]:
!pip install onnx==1.4.1

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [55]:
import onnx
import caffe2.python.onnx.backend as onnx_caffe2_backend

## Load the model
*  prepare the caffe2 backend for executing the model. this converts the ONNX model into a Caffe2 NetDef that can execute it.
* The graph of the model itself contains inputs for all weight parameters

In [56]:
new_model = onnx.load("classifier.onnx")

In [57]:
prepared_backend = onnx_caffe2_backend.prepare(new_model)

In [58]:
new_model.graph.input[0]

name: "input.1"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 72
      }
      dim {
        dim_value: 13
      }
    }
  }
}

#### Get the prediction from the loaded model

In [59]:
test_df = pd.read_csv("data/wine_test_data.csv")

In [60]:
test_df

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,11.65,1.67,2.62,26.0,88,1.92,1.61,0.40,1.34,2.60,1.36,3.21,562
2,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
3,2,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
4,2,12.82,3.37,2.30,19.5,88,1.48,0.66,0.40,0.97,10.26,0.72,1.75,685
5,0,12.93,3.80,2.65,18.6,102,2.41,2.41,0.25,1.98,4.50,1.03,3.52,770
6,1,11.84,2.89,2.23,18.0,112,1.72,1.32,0.43,0.95,2.65,0.96,2.52,500
7,2,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840
8,1,11.41,0.74,2.50,21.0,88,2.48,2.01,0.42,1.44,3.08,1.10,2.31,434
9,1,12.08,1.33,2.30,23.6,70,2.20,1.59,0.42,1.38,1.74,1.07,3.21,625


In [61]:
x_test = test_df.drop('Class', axis = 1)
y_test = test_df['Class']

#### Converting the features to a numpy array and then a tensor

In [62]:
sample = np.array(x_test)

sample_tensor = torch.from_numpy(sample).float()

In [63]:
sample_tensor

tensor([[1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03],
        [1.1650e+01, 1.6700e+00, 2.6200e+00, 2.6000e+01, 8.8000e+01, 1.9200e+00,
         1.6100e+00, 4.0000e-01, 1.3400e+00, 2.6000e+00, 1.3600e+00, 3.2100e+00,
         5.6200e+02],
        [1.4370e+01, 1.9500e+00, 2.5000e+00, 1.6800e+01, 1.1300e+02, 3.8500e+00,
         3.4900e+00, 2.4000e-01, 2.1800e+00, 7.8000e+00, 8.6000e-01, 3.4500e+00,
         1.4800e+03],
        [1.3270e+01, 4.2800e+00, 2.2600e+00, 2.0000e+01, 1.2000e+02, 1.5900e+00,
         6.9000e-01, 4.3000e-01, 1.3500e+00, 1.0200e+01, 5.9000e-01, 1.5600e+00,
         8.3500e+02],
        [1.2820e+01, 3.3700e+00, 2.3000e+00, 1.9500e+01, 8.8000e+01, 1.4800e+00,
         6.6000e-01, 4.0000e-01, 9.7000e-01, 1.0260e+01, 7.2000e-01, 1.7500e+00,
         6.8500e+02],
        [1.2930e+01, 3.8000e+00, 2.6500e+00, 1.8600e+01, 1.0200e+02, 2.4100e+00,

In [64]:
sample = np.array(x_test) 

sample_tensor = torch.from_numpy(sample).float()

## Run the model
* Since the weights are already embedded, we just need to pass the input

In [65]:
W = {new_model.graph.input[0].name: sample_tensor .numpy()}

In [66]:
print(W)

{'input.1': array([[1.423e+01, 1.710e+00, 2.430e+00, 1.560e+01, 1.270e+02, 2.800e+00,
        3.060e+00, 2.800e-01, 2.290e+00, 5.640e+00, 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.165e+01, 1.670e+00, 2.620e+00, 2.600e+01, 8.800e+01, 1.920e+00,
        1.610e+00, 4.000e-01, 1.340e+00, 2.600e+00, 1.360e+00, 3.210e+00,
        5.620e+02],
       [1.437e+01, 1.950e+00, 2.500e+00, 1.680e+01, 1.130e+02, 3.850e+00,
        3.490e+00, 2.400e-01, 2.180e+00, 7.800e+00, 8.600e-01, 3.450e+00,
        1.480e+03],
       [1.327e+01, 4.280e+00, 2.260e+00, 2.000e+01, 1.200e+02, 1.590e+00,
        6.900e-01, 4.300e-01, 1.350e+00, 1.020e+01, 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.282e+01, 3.370e+00, 2.300e+00, 1.950e+01, 8.800e+01, 1.480e+00,
        6.600e-01, 4.000e-01, 9.700e-01, 1.026e+01, 7.200e-01, 1.750e+00,
        6.850e+02],
       [1.293e+01, 3.800e+00, 2.650e+00, 1.860e+01, 1.020e+02, 2.410e+00,
        2.410e+00, 2.500e-01, 1.980e+00, 4.500e+00, 1.030e+00, 3.520e+00,


In [67]:
c2_out = prepared_backend.run(W)

In [68]:
c2_out

Outputs(_0=array([[-1.5628026e-03, -6.4632020e+00, -1.3248754e+01],
       [-6.6571174e+00, -1.3892760e-03, -9.1768579e+00],
       [-2.0265237e-03, -1.1226930e+01, -6.2090254e+00],
       [-2.8680639e+00, -8.8724508e+00, -5.8634937e-02],
       [-2.9358292e+00, -8.8798733e+00, -5.4694731e-02],
       [-2.3397297e-01, -1.6491035e+00, -4.1108022e+00],
       [-4.6636252e+00, -1.6491409e-02, -4.9727721e+00],
       [-2.8658440e+00, -8.8712769e+00, -5.8768939e-02],
       [-6.5891380e+00, -1.4955255e-03, -9.0341454e+00],
       [-2.1384964e+00, -1.2545919e-01, -9.4798193e+00]], dtype=float32))

In [69]:
c2_out[0]

array([[-1.5628026e-03, -6.4632020e+00, -1.3248754e+01],
       [-6.6571174e+00, -1.3892760e-03, -9.1768579e+00],
       [-2.0265237e-03, -1.1226930e+01, -6.2090254e+00],
       [-2.8680639e+00, -8.8724508e+00, -5.8634937e-02],
       [-2.9358292e+00, -8.8798733e+00, -5.4694731e-02],
       [-2.3397297e-01, -1.6491035e+00, -4.1108022e+00],
       [-4.6636252e+00, -1.6491409e-02, -4.9727721e+00],
       [-2.8658440e+00, -8.8712769e+00, -5.8768939e-02],
       [-6.5891380e+00, -1.4955255e-03, -9.0341454e+00],
       [-2.1384964e+00, -1.2545919e-01, -9.4798193e+00]], dtype=float32)

In [70]:
output_tensor = torch.Tensor(c2_out[0])

In [71]:
output_tensor

tensor([[-1.5628e-03, -6.4632e+00, -1.3249e+01],
        [-6.6571e+00, -1.3893e-03, -9.1769e+00],
        [-2.0265e-03, -1.1227e+01, -6.2090e+00],
        [-2.8681e+00, -8.8725e+00, -5.8635e-02],
        [-2.9358e+00, -8.8799e+00, -5.4695e-02],
        [-2.3397e-01, -1.6491e+00, -4.1108e+00],
        [-4.6636e+00, -1.6491e-02, -4.9728e+00],
        [-2.8658e+00, -8.8713e+00, -5.8769e-02],
        [-6.5891e+00, -1.4955e-03, -9.0341e+00],
        [-2.1385e+00, -1.2546e-01, -9.4798e+00]])

In [72]:
_, predicted = torch.max(output_tensor.data, -1)

In [73]:
predicted

tensor([0, 1, 0, 2, 2, 0, 1, 2, 1, 1])

In [74]:
predicted_df = pd.DataFrame(data = predicted.numpy(), columns = ['Predicted Class'])

In [75]:
compare = [predicted_df, y_test]
comapare_df = pd.concat(compare, axis=1)

In [76]:
comapare_df

,Predicted Class,Class
0,0,0
1,1,1
2,0,0
3,2,2
4,2,2
5,0,0
6,1,1
7,2,2
8,1,1
9,1,1
